[![alt text](https://avatars1.githubusercontent.com/u/59831504?s=400&v=4 "MtheEPIC User Icon")](https://github.com/MtheEPIC/KaggleProj)

# Import libs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.axes_style("whitegrid")

import glob

from sklearn.decomposition import PCA

from sklearn.preprocessing import MinMaxScaler, binarize
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, plot_roc_curve, confusion_matrix, classification_report, precision_recall_curve, average_precision_score  
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, cross_val_predict

from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC

# Paths and Consts

In [2]:
r_hand_path = 'data\\HandRight.csv'
path = 'data\\Training\\*\\*'
start_delay = 7

# Funcs

In [60]:
def add_suffix_on_split(split_df1, split_df2):
    if split_df1['Hand Type'][0] == 'left':
        split_df1 = split_df1.add_suffix(' Left')
        split_df2 = split_df2.add_suffix(' Right')
    else:
        split_df1 = split_df1.add_suffix(' Right')
        split_df2 = split_df2.add_suffix(' Left')
    return split_df1, split_df2

In [61]:
def combine_hands(df, split_df2=None):
    if split_df2 is None:
        split1 = np.arange(0, df.shape[0], 2)
        split2 = np.arange(1, df.shape[0], 2)
        
        split_df1 = df.iloc[split1, :]
        split_df2 = df.iloc[split2, :]
    else:
        split_df1 = df.copy()

    if split_df1.shape[0] > split_df2.shape[0]:
        split_df1 = split_df1.iloc[np.arange(split_df2.shape[0]), :]
    else:
        split_df2 = split_df2.iloc[np.arange(split_df1.shape[0]), :]

    split_df1.set_index(np.arange(split_df1.shape[0]), inplace=True)
    split_df2.set_index(np.arange(split_df1.shape[0]), inplace=True)
    
    split_df1, split_df2 = add_suffix_on_split(split_df1, split_df2)

    df = pd.concat([split_df1, split_df2], axis=1, sort=False)
    return df

In [43]:
def mod_df(df, df2=None):
    df.rename(str.strip, axis='columns', inplace=True)
#     df.drop('# hands', axis=1, inplace=True)
#     df = df.replace(['left', 'right'], [0, 1])
    df = df[df['Time']>=df['Time'][0]+start_delay]
    
    
    start_time = df.iloc[0, 0]
    end_time = df.iloc[len(df)-1, 0]
    total_time = (end_time - start_time)
    start_frame = df.iloc[0, 1]
    end_frame = df.iloc[len(df)-1, 1]
    total_frame = (end_frame - start_frame)
    
#     tmp1 = np.arange(0, df.shape[0], int(total_time))
#     tmp2 = np.arange(1, df.shape[0], int(total_time))
#     df['tmp'] = np.arange(df.shape[0])
#     df_cut = df[df['tmp'].isin(np.append(tmp1, tmp2))]
#     df_cut.drop('tmp', axis=1, inplace=True)
    
    df_cut = df
#     print(df_cut)
    
    df_cut = combine_hands(df_cut, df2)
    
    
#     jump = total_frame//100
#     print(start_time, ' ', end_time, ' ', jump, ' ', start_frame, ' ', end_frame)
    
#     print(df_cut.head())
    return df_cut

In [44]:
# tmp = mod_df(train_spontan)
# # train_spontan.iloc[0, 0]
# tmp.info()
# tmp.head(22)

In [45]:
# train_sync = pd.read_csv('data/Training/Evyatar Cohen/Evyatar636771050902934708Sync.csv')
# train_sync.drop(' # hands', axis=1, inplace=True)
# train_sync = train_sync.replace(['left', 'right'], [0, 1])
# print(train_sync.columns)
# train_sync.head()

In [8]:
# train_sync.info()
# train_sync.describe()

In [9]:
# # After big join

# from sklearn import preprocessing
# mm_scaler = preprocessing.MinMaxScaler()
# X_train_minmax = mm_scaler.fit_transform(train_sync)
# tmp = mm_scaler.transform(train_sync)
# tmpdf = pd.DataFrame(tmp)
# # tmpdf = tmpdf.rename(columns=(np.arange(1, 22)))
# for i in range(tmpdf.shape[1]):
#     tmpdf.rename(columns={i: train_sync.columns[i]}, inplace=True)
# tmpdf

## data mod

In [10]:
# train_spontan.iloc[0, 0]

In [11]:
# train_spontan['Time']

In [12]:
# from os import listdir
# from os.path import isfile, join

# mypath = os.path.dirname(os.path.realpath(__file__))
# print(mypath)
# # onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [62]:
cnt = 0

train_df = pd.DataFrame()

r_hand_df = pd.read_csv(r_hand_path)
r_hand_df.dropna(inplace=True)
r_hand_df.rename(str.strip, axis='columns', inplace=True)
r_hand_df = r_hand_df[r_hand_df['Time']>=r_hand_df['Time'][0]+start_delay]
# r_hand_df['value'] = 0
print(r_hand_df.shape)

for name in glob.glob(path):
    print(name)
    df = pd.read_csv(name)
#     print('origen', df.shape[0])
    
    df.dropna(inplace=True)
    if 'Alone' in name[-11:]: #add r hand
        df = mod_df(df, r_hand_df.copy())
        
        
        df['value'] = 0
        
        train_df = train_df.append(df, ignore_index=True)
        
    else:
        df  = mod_df(df)
        
        if 'Spontan' in name[-11:]:
            df['value'] = 1
        elif 'Sync' in name[-11:]:
            df['value'] = 2
        else:
            print('ERORR')
        
        train_df = train_df.append(df, ignore_index=True)
        
    print('after', df.shape)
#     train_df = train_df.append(df, ignore_index=True)
#     print(df.head())
cnt

(4007, 22)
data\Training\Evyatar Cohen\Evyatar636771050011691836Spontan.csv
after (4104, 45)
data\Training\Evyatar Cohen\Evyatar636771050902934708Sync.csv
after (4104, 45)
data\Training\Evyatar Cohen\Evyatar636771051823980826Alone.csv
after (4007, 45)
data\Training\Evyatar Cohen\Evyatar636771052727603804Spontan.csv
after (4104, 45)
data\Training\Evyatar Cohen\Evyatar636771053639929594Sync.csv
after (4103, 45)
data\Training\Evyatar Cohen\Evyatar636771054555711409Alone.csv
after (4007, 45)
data\Training\Nofar Social_Nuero\Nofar636759792183061005Spontan.csv
after (4104, 45)
data\Training\Nofar Social_Nuero\Nofar636759793193137655Alone.csv
after (4007, 45)
data\Training\Nofar Social_Nuero\Nofar636759794200223541Sync.csv
after (4103, 45)
data\Training\Nofar Social_Nuero\Nofar636759795182793299Spontan.csv
after (4076, 45)
data\Training\Nofar Social_Nuero\Nofar636759796290435160Alone.csv
after (4007, 45)
data\Training\Nofar Social_Nuero\Nofar636759797397919664Sync.csv
after (4104, 45)
data\Tr

0

In [63]:
train_df

,Time Left,Frame ID Left,Hand Type Left,# hands Left,Position X Left,Position Y Left,Position Z Left,Velocity X Left,Velocity Y Left,Velocity Z Left,...,Wrist Pos X Right,Wrist Pos Y Right,Wrist Pos Z Right,Elbow pos X Right,Elbow Pos Y Right,Elbow Pos Z Right,Grab Strenth Right,Grab Angle Right,Pinch Strength Right,value
0,71.41422,98073,left,2,-109.80610,228.3820,87.325780,-41.53802,116.70710,-60.09788,...,172.2568,166.6150,15.43945,313.4828,3.741617,144.85630,0.0,0.229723,0.0,1
1,71.43108,98075,left,2,-110.42900,230.8011,86.419320,-23.84248,125.97130,-58.77163,...,172.4875,167.4718,16.26752,312.2578,1.103906,142.78720,0.0,0.233833,0.0,1
2,71.44753,98076,left,2,-110.42230,231.4766,85.786140,0.77496,77.71951,-72.85492,...,173.3618,168.3325,16.97992,313.3397,0.328209,141.08470,0.0,0.229966,0.0,1
3,71.46376,98078,left,2,-110.69090,233.3635,84.411220,-22.08076,105.68630,-83.23758,...,174.7678,170.0557,18.96619,318.9720,0.158455,135.41930,0.0,0.278752,0.0,1
4,71.48033,98080,left,2,-111.44740,236.8768,83.475110,-54.70446,234.09310,-51.02924,...,176.5129,171.9669,21.02561,325.5248,-1.036881,126.31750,0.0,0.231410,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177678,856.19650,67304,left,2,-77.62601,173.7391,-14.213500,-46.75000,99.50911,71.01207,...,129.1261,148.7041,62.82404,300.6781,-33.586720,128.75430,0.0,0.000000,0.0,2
177679,856.22910,67306,left,2,-78.43058,176.0170,-12.152660,-33.69026,98.24361,59.77054,...,129.2868,148.1146,63.11218,285.6505,-53.666400,106.02620,0.0,0.000000,0.0,2
177680,856.26310,67309,left,2,-79.84506,180.2403,-9.015577,-68.45120,136.37580,87.63277,...,131.8590,149.6264,66.09853,283.6185,-58.034910,95.29868,0.0,0.000000,0.0,2
177681,856.29580,67311,left,2,-81.57359,183.5054,-7.185198,-63.24731,121.75270,62.95970,...,134.0416,151.5924,68.17693,288.9688,-54.138200,94.23959,0.0,0.000000,0.0,2


In [64]:
train_df.isna().sum()

Time Left               0
Frame ID Left           0
Hand Type Left          0
# hands Left            0
Position X Left         0
Position Y Left         0
Position Z Left         0
Velocity X Left         0
Velocity Y Left         0
Velocity Z Left         0
Pitch Left              0
Roll Left               0
Yaw Left                0
Wrist Pos X Left        0
Wrist Pos Y Left        0
Wrist Pos Z Left        0
Elbow pos X Left        0
Elbow Pos Y Left        0
Elbow Pos Z Left        0
Grab Strenth Left       0
Grab Angle Left         0
Pinch Strength Left     0
Time Right              0
Frame ID Right          0
Hand Type Right         0
# hands Right           0
Position X Right        0
Position Y Right        0
Position Z Right        0
Velocity X Right        0
Velocity Y Right        0
Velocity Z Right        0
Pitch Right             0
Roll Right              0
Yaw Right               0
Wrist Pos X Right       0
Wrist Pos Y Right       0
Wrist Pos Z Right       0
Elbow pos X 

In [65]:
train_df.columns

Index(['Time Left', 'Frame ID Left', 'Hand Type Left', '# hands Left',
       'Position X Left', 'Position Y Left', 'Position Z Left',
       'Velocity X Left', 'Velocity Y Left', 'Velocity Z Left', 'Pitch Left',
       'Roll Left', 'Yaw Left', 'Wrist Pos X Left', 'Wrist Pos Y Left',
       'Wrist Pos Z Left', 'Elbow pos X Left', 'Elbow Pos Y Left',
       'Elbow Pos Z Left', 'Grab Strenth Left', 'Grab Angle Left',
       'Pinch Strength Left', 'Time Right', 'Frame ID Right',
       'Hand Type Right', '# hands Right', 'Position X Right',
       'Position Y Right', 'Position Z Right', 'Velocity X Right',
       'Velocity Y Right', 'Velocity Z Right', 'Pitch Right', 'Roll Right',
       'Yaw Right', 'Wrist Pos X Right', 'Wrist Pos Y Right',
       'Wrist Pos Z Right', 'Elbow pos X Right', 'Elbow Pos Y Right',
       'Elbow Pos Z Right', 'Grab Strenth Right', 'Grab Angle Right',
       'Pinch Strength Right', 'value'],
      dtype='object')

In [66]:
train_df.nunique()

Time Left               175008
Frame ID Left            95795
Hand Type Left               2
# hands Left                 2
Position X Left         174046
Position Y Left         169344
Position Z Left         176512
Velocity X Left         176841
Velocity Y Left         176732
Velocity Z Left         176899
Pitch Left              172186
Roll Left               175461
Yaw Left                172855
Wrist Pos X Left        171553
Wrist Pos Y Left        167723
Wrist Pos Z Left        176249
Elbow pos X Left        172206
Elbow Pos Y Left        176910
Elbow Pos Z Left        175239
Grab Strenth Left        25473
Grab Angle Left         127099
Pinch Strength Left      41343
Time Right              128396
Frame ID Right           81203
Hand Type Right              2
# hands Right                2
Position X Right        125484
Position Y Right        125152
Position Z Right        128880
Velocity X Right        129056
Velocity Y Right        129006
Velocity Z Right        129123
Pitch Ri

In [24]:
tmp = tmp.set_index(np.arange(0, tmp.shape[0]*2, 2))
tmp

NameError: name 'tmp' is not defined

In [ ]:
# r_hand_df = pd.read_csv(r_hand_path)
# rrrr = r_hand_df
r_hand_df

In [ ]:
r_hand_df = r_hand_df.set_index(np.arange(1, r_hand_df.shape[0]*2, 2))
r_hand_df

In [ ]:
r_hand_df.join(tmp, how='outer')
r_hand_df

In [ ]:
train_sync

In [ ]:
train_sync = pd.read_csv('data/Training/Evyatar Cohen/Evyatar636771050902934708Sync.csv')
# train_sync = train_sync.astype({' Frame ID': 'object'}).dtypes
# train_sync = train_sync.groupby(' Frame ID').agg(lambda x: pd.concat([x], axis=1, sort=False))
# train_sync.set_index(' Frame ID', inplace=True)
# pd.concat([], axis=1, sort=False))
split1 = np.arange(0, train_sync.shape[0], 2)
split2 = np.arange(1, train_sync.shape[0], 2)
df1 = train_sync.iloc[split1, :]
df2 = train_sync.iloc[split2, :]
df1.set_index(np.arange(df1.shape[0]), inplace=True)
df2.set_index(np.arange(df2.shape[0]), inplace=True)
if df1[' Hand Type'][0] == 'left':
    df1.add_suffix(' Left')
    df2.add_suffix(' Right')
else:
    df1 = df1.add_suffix(' Right')
    df2.add_suffix(' Left')
train_sync
df3 = pd.concat([df1, df2], axis=1, sort=False)
df3